# Table of Contents
 <p><div class="lev2 toc-item"><a href="#1主要步骤" data-toc-modified-id="1主要步骤-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>1主要步骤</a></div><div class="lev2 toc-item"><a href="#2模拟微博登录" data-toc-modified-id="2模拟微博登录-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>2模拟微博登录</a></div><div class="lev2 toc-item"><a href="#3获得并解析json内容" data-toc-modified-id="3获得并解析json内容-03"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>3获得并解析json内容</a></div><div class="lev2 toc-item"><a href="#4爬取并存入mysql数据库" data-toc-modified-id="4爬取并存入mysql数据库-04"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>4爬取并存入mysql数据库</a></div>

## 1主要步骤

* 模拟微博登录
* 解析json内容页
* 将爬取内容存进mysql数据库

## 2模拟微博登录

In [3]:
#python3.5相关包的导入
from pandas import Series
from pandas import DataFrame
import requests
import json
import re
import time as ti
import MySQLdb 
import http.cookiejar as cookielib
from pandas import to_datetime

模拟微博登录请求的headers配置，首先要在chrome浏览器上登录移动端的微博http://m.weibo.cn   
登录自己的账号密码，打开开发者工具，勾选Preserve log查看下面的header来自Network-->Name-->login--->Request Headers，如下图  
![login图片](./相关图片/login页面.png)   
从这里获得需要的Request Headers和Form Data的相关信息为模拟登录提供必要的信息


In [7]:
headers={"Accept":"*/*",
         "Accept-Encoding":"gzip, deflate, br",
         "Accept-Language":"zh-CN,zh;q=0.8",
         "Connection":"keep-alive",
         "Content-Length":"294",
         "Content-Type":"application/x-www-form-urlencoded",
         "Cookie":"_T_WM=87bffde59b3a3d7afe90e24876aa6cd8",
         "Host":"passport.weibo.cn",
         "Origin":"https://passport.weibo.cn",
         "Referer":"https://passport.weibo.cn/signin/login?entry=mweibo&res=wel&wm=3349&r=http%3A%2F%2Fm.weibo.cn%2F",
         "User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36"

         }

fromData={
          "username":"账号",
          "password":"密码",
          "savestate":"1",
          "r":"http://m.weibo.cn/",
          "ec":"0",
          "pagerefer":"https://passport.weibo.cn/signin/welcome?entry=mweibo&r=http%3A%2F%2Fm.weibo.cn%2F",
          "entry":"mweibo",
          "mainpageflag":"1"
        
          }
pageURL="https://passport.weibo.cn/sso/login"
session = requests.session()
session.cookies = cookielib.LWPCookieJar(filename='cookies')
login_page = session.post(pageURL, data=fromData, headers=headers)

## 3获得并解析json内容

点击刷新当前页面，找到json内容的url   
![json1](./相关图片/json1.png)   
打开该json的url获得用来翻看内容的next_cursor和page（默认值为1）   
![json2](./相关图片/json2.png)   
实际上这个json内容网页的整个url为 http://m.weibo.cn/feed/friends?version=v4&next_cursor=4096757350386351&page=1   
下面为相关处理函数

In [8]:
#使发表时间按照统一的字符串格式输出
def format_time(time_string):
    ti_string=ti.strftime("%H:%M",ti.localtime())
    (hour,minute)=ti_string.split(':')
    hour=int(hour)
    minute=int(minute)
    if '分钟前' in time_string:
        result = re.findall(r"[0-9]",time_string)
        l=len(result)
        if l==1:
            diff=int(result[0])
            if minute>=diff:
                new_minute=minute-diff
                return(ti.strftime("%m-%d ",ti.localtime())+str(hour)+':'+str(new_minute))                                                                     
            else:
                new_hour=hour-1
                new_minute=60+minute-diff
                return(ti.strftime("%m-%d ",ti.localtime())+str(new_hour)+':'+str(new_minute))
        else:
            diff=str(result[0])+str(result[1])
            diff=int(diff)
            if minute>=diff:
                new_minute=minute-diff
                return(ti.strftime("%m-%d ",ti.localtime())+str(hour)+':'+str(new_minute))                                                                     
            else:
                new_hour=hour-1
                new_minute=60+minute-diff
                return(ti.strftime("%m-%d ",ti.localtime())+str(new_hour)+':'+str(new_minute))
    elif  '今天' in time_string:
        return(time_string.replace('今天',ti.strftime("%m-%d",ti.localtime(ti.time()))))
    else:
        return(time_string) 
        
#微博发表的时间段
def get_hour(time_string):
    f_date=to_datetime('2017-'+time_string,format='%Y-%m-%d %H:%M')
    hour_num=f_date.hour
    return(hour_num)

#获得某个数据库的某个数据表当前的记录数，返回值为int类型
def get_records_num(conn,table_name):
    cursor=conn.cursor()
    sql=("""SELECT count(*) FROM  %s"""%(table_name))
    cursor.execute(sql)
    records_num=cursor.fetchone()[0]
    return(records_num)     

#拼接demo_page的url
def joinURL(now_cursor,pageNum):
    demo_pageURL="http://m.weibo.cn/feed/friends?version=v4&next_cursor="+str(now_cursor)+"&page="+str(pageNum)
    return(demo_pageURL)
    
#获取某页的card的内容，并格式化为dataframe
def get_card_group(pageNum,jsonObject,conn,table_name):
    record_count=get_records_num(conn,table_name)
    data=DataFrame(columns=['record_count','hourNum','pageNum','userID','nickName','user_description','user_fansNum','time','content','reposts_count'])
    counts=len(jsonObject[0]['card_group'])
    for each_count in range(0,counts):
        record_count=record_count+1
        id=str(jsonObject[0]['card_group'][each_count]['mblog']['id'])
        time=format_time(jsonObject[0]['card_group'][each_count]['mblog']['created_at'])
        hour_num=get_hour(time)
        screen_name=jsonObject[0]['card_group'][each_count]['mblog']['user']        ['screen_name']
        description=jsonObject[0]['card_group'][each_count]['mblog']['user']['description']
        fansNum=jsonObject[0]['card_group'][each_count]['mblog']['user']['follow_count']
        reposts_count=str(jsonObject[0]['card_group'][each_count]['mblog']['reposts_count'])
        if 'retweeted_status' in str(jsonObject[0]['card_group'][each_count]['mblog']):
            content=str(jsonObject[0]['card_group'][each_count]['mblog']['text'])+'---转发自---'+str(jsonObject[0]['card_group'][each_count]['mblog']['retweeted_status']['text'])
        else:
            content=str(jsonObject[0]['card_group'][each_count]['mblog']['text'])
    
        data=data.append(
                             Series(
                [record_count,hour_num,pageNum,id,screen_name,description,fansNum,time,content,reposts_count], 
                index=['record_count','hourNum','pageNum','userID','nickName','user_description','user_fansNum','time','content','reposts_count']
            ), ignore_index=True
                                    )
    return(data)

## 4爬取并存入mysql数据库

In [9]:
#建立和数据库的连接
conn = MySQLdb.connect(
            host='127.0.0.1',
            user='root',
            passwd='admin',
            port=3306,
            db='weibo',
            charset='utf8'
        )

In [10]:
total_page=2
now_cursor=4096757350386351
for pageNum in range(1,total_page+1):
    demo_page = session.get(joinURL(now_cursor,pageNum))  
    jsonString =demo_page.text 
    jsonObject = json.loads(jsonString)
    df=get_card_group(pageNum,jsonObject,conn,'demo') 
    df.to_sql(name='demo', con=conn, flavor='mysql', if_exists='append', index=False)
    ti.sleep(2)
    print('*********************第'+str(pageNum)+'页的微博已存入数据库***********************')
    if jsonObject[0]['next_cursor']==0:
        print('end with pageNum='+str(pageNum-1))
        break

D:\Anaconda3\AZ\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)
D:\Anaconda3\AZ\lib\site-packages\pandas\io\sql.py:1445: Warning: Incorrect string value: '\xF0\x9F\x99\x8F--...' for column 'content' at row 6
  conn.executemany(self.insert_statement(), data_list)


*********************第1页的微博已存入数据库***********************
*********************第2页的微博已存入数据库***********************


In [14]:
#DataFrame的内容为
df.head(2)

,record_count,hourNum,pageNum,userID,nickName,user_description,user_fansNum,time,content,reposts_count
0,675.0,8.0,2.0,4096706754650692,我们爱看买家秀,只为传递快乐！欢迎投稿&amp;吐槽,54.0,04-15 08:01,"王者荣耀手办，我是拒绝的<span class=""iconimg iconimg-xs""><...",513
1,676.0,8.0,2.0,4096706595272234,中国青年报,历史的一份底稿。（中国青年报官方网站中青在线 www.cyol.com 微信公众账号：zqb...,1391.0,04-15 08:00,【“天价彩礼”的苦 国外男青年也懂】“天价”让彩礼变了味儿，本该喜庆的事情成了一桩讨价还价的...,17


mysql数据库中，用可视化工具Navicat看到   
![mysql数据表](./相关图片/mysql数据表.png)

p1.py为代码   
转载请注明出处，谢谢！ 